In [ ]:
!pip install diffusers
!pip install transformers
!pip install accelerate
!pip install ftfy

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from PIL import Image
from torchvision import models, transforms, utils as vutils
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib
from torch.utils.data import DataLoader, Dataset, Subset
from diffusers import DiffusionPipeline
import matplotlib.pyplot as plt
import json  


In [ ]:


# Verifica delle GPU disponibili
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if device == "cuda" else torch.float32

# Caricamento della pipeline Stable Diffusion
pipeline = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1", torch_dtype=dtype)
pipeline.to(device)

# Percorso al file JSON delle annotazioni COCO (modifica il percorso in base alla tua configurazione)
coco_annotation_file = '/kaggle/input/coco-2017-dataset/coco2017/annotations/instances_train2017.json'

# Funzione per leggere il file JSON delle annotazioni
def load_coco_annotations(json_file):
    """
    Carica le annotazioni dal file JSON del dataset COCO.
    
    Args:
    - json_file (str): Il percorso del file JSON con le annotazioni COCO.

    Returns:
    - dict: Un dizionario con i dati delle annotazioni COCO.
    """
    with open(json_file, 'r') as f:
        data = json.load(f)
    return data

# Carica le annotazioni COCO
coco_data = load_coco_annotations(coco_annotation_file)

# Mappa degli ID delle categorie alle descrizioni delle categorie
categories = {cat['id']: cat['name'] for cat in coco_data['categories']}

# Creazione delle directory per salvare le immagini 64x64, 256x256 e originali localmente su Kaggle
base_dir = '/kaggle/working/Immagini_stable/'
dir_64x64 = os.path.join(base_dir, '64x64')
dir_256x256 = os.path.join(base_dir, '256x256')
dir_original = os.path.join(base_dir, 'original')

# Creazione delle directory se non esistono già
os.makedirs(dir_64x64, exist_ok=True)
os.makedirs(dir_256x256, exist_ok=True)
os.makedirs(dir_original, exist_ok=True)

# Estrazione delle annotazioni per le immagini
image_annotations = {}
for annotation in coco_data['annotations']:
    img_id = annotation['image_id']
    category_name = categories[annotation['category_id']]
    
    if img_id in image_annotations:
        image_annotations[img_id].append(category_name)
    else:
        image_annotations[img_id] = [category_name]

# Selezione delle seconde 2000 immagini
sample_ids = list(image_annotations.keys())[2000:4000]  # Seleziona le immagini dal 2001° al 4000°

# Funzione per generare immagini a partire da un prompt
def generate_image(prompt):
    """
    Genera un'immagine utilizzando la pipeline Stable Diffusion a partire da un prompt.
    
    Args:
    - prompt (str): Il prompt testuale per generare l'immagine.

    Returns:
    - image: L'immagine generata come oggetto PIL.Image.
    """
    image = pipeline(prompt, num_inference_steps=8).images[0]
    return image.convert("RGB")

# Iterazione per generare le seconde 2000 immagini e salvarle
for i, img_id in enumerate(sample_ids, start=1):  # Inizia la numerazione da 1
    try:
        # Crea il prompt dalle categorie associate a questa immagine
        prompt = ", ".join(image_annotations[img_id])

        # Genera immagine fake usando il prompt
        image_fake = generate_image(prompt)

        # Ridimensiona l'immagine a 64x64 e 256x256
        image_64x64 = image_fake.resize((64, 64))
        image_256x256 = image_fake.resize((256, 256))

        # Percorsi per salvare le immagini nelle sottocartelle locali
        save_path_64 = f'{dir_64x64}/fake_image_{i}.png'
        save_path_256 = f'{dir_256x256}/fake_image_{i}.png'
        save_path_original = f'{dir_original}/image_{i}.png'  # Nome immagine originale con numerazione da 1

        # Salva le immagini nelle rispettive directory
        image_64x64.save(save_path_64)
        image_256x256.save(save_path_256)
        image_fake.save(save_path_original)  # Salva l'immagine originale come image_{i}.png

        print(f"Immagine {i}/2000 generata e salvata in {save_path_64}, {save_path_256}, e {save_path_original}")

    except Exception as e:
        print(f"Errore nella generazione dell'immagine ID {img_id}: {e}")


In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))
download_file('/kaggle/working/Immagini_stable', 'out')
#download_file('/kaggle/working/Immagini_gan', 'out1')
#download_file('/kaggle/working/IS_FIS_results', 'out2')

In [ ]:
import shutil
import os

# Percorso della cartella da eliminare
base_dir = '/kaggle/working/metrics_combined_64x64_compression_60.txt'

# Controlla se la cartella esiste, quindi eliminala insieme a tutti i suoi contenuti
if os.path.exists(base_dir):
    shutil.rmtree(base_dir)
    print(f"Tutto il contenuto della cartella {base_dir} è stato eliminato.")
else:
    print(f"La cartella {base_dir} non esiste.")


In [ ]:


# Iperparametri per il Generatore
image_size = 64  # Dimensione delle immagini generate
nz = 100  # Dimensione del vettore latente
ngf = 64  # Numero di feature maps nel generatore
nc = 3  # Numero di canali nelle immagini di allenamento (RGB)
ngpu = torch.cuda.device_count()  # Rileva automaticamente il numero di GPU disponibili

# Definizione del modello Generator
class Generator(nn.Module):
    """
    Una classe che rappresenta il modello del Generatore per la generazione di immagini.
    Il modello utilizza una serie di layer ConvTranspose2d per upscalare il vettore latente 
    fino a ottenere la dimensione dell'immagine desiderata.

    Args:
    - ngpu (int): Numero di GPU disponibili per il calcolo parallelo.
    """
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),  # Layer 1
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),  # Layer 2
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),  # Layer 3
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),  # Layer 4
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),  # Layer finale
            nn.Tanh()  # Output scalato tra -1 e 1
        )

    def forward(self, input):
        """
        Passaggio forward del Generatore. Riceve un vettore latente e produce un'immagine.
        
        Args:
        - input (Tensor): Un batch di vettori latenti con forma [batch_size, nz, 1, 1].

        Returns:
        - Tensor: Immagini generate.
        """
        return self.main(input)

# Inizializzazione del generatore
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
netG = Generator(ngpu=ngpu)  # Creazione di un'istanza del modello Generator

# Usa DataParallel se sono disponibili più GPU
if ngpu > 1:
    netG = nn.DataParallel(netG, list(range(ngpu)))

netG = netG.to(device)  # Sposta il modello su GPU (o CPU se non disponibile)

# Carica i pesi del generatore da un checkpoint
checkpoint_path_generator = '/kaggle/input/checkpoint-generatore/checkpoint_generator.pth'
state_dict = torch.load(checkpoint_path_generator, map_location=device)

# Controlla se lo state_dict proviene da un modello DataParallel e modifica se necessario
new_state_dict = {}
for key, value in state_dict.items():
    if ngpu > 1 and not key.startswith("module."):
        new_state_dict["module." + key] = value  # Aggiungi 'module.' come prefisso se si usa DataParallel
    elif ngpu == 1 and key.startswith("module."):
        new_state_dict[key[len("module."):]] = value  # Rimuovi 'module.' se non si usa DataParallel
    else:
        new_state_dict[key] = value

# Carica il nuovo state dict modificato nel modello
netG.load_state_dict(new_state_dict)

# Imposta il generatore in modalità di valutazione
netG.eval()

# Genera vettori latenti (rumore casuale)
fixed_noise = torch.randn(2000, nz, 1, 1, device=device)  # Genera 2000 vettori di rumore casuale

# Genera 2000 immagini fake dal generatore
with torch.no_grad(): 
    fake_images = netG(fixed_noise).detach().cpu()  # Genera immagini e spostale su CPU

# Imposta la directory per salvare le immagini generate
save_directory = '/kaggle/working/Immagini_gan/'
dir_64x64 = os.path.join(save_directory, '64x64')
dir_256x256 = os.path.join(save_directory, '256x256')

# Crea le directory se non esistono
os.makedirs(dir_64x64, exist_ok=True)
os.makedirs(dir_256x256, exist_ok=True)

# Salva ogni immagine generata in versioni 64x64 e 256x256
for i in range(fake_images.size(0)):
    image_64_path = os.path.join(dir_64x64, f'generated_image_{i+1}.png')
    image_256_path = os.path.join(dir_256x256, f'generated_image_{i+1}.png')

    # Salva la versione 64x64
    vutils.save_image(fake_images[i], image_64_path, normalize=True)

    # Ridimensiona l'immagine a 256x256 e salvala
    image_256 = F.interpolate(fake_images[i].unsqueeze(0), size=(256, 256)).squeeze(0)
    vutils.save_image(image_256, image_256_path, normalize=True)

    print(f"Immagini salvate: {image_64_path}, {image_256_path}")


In [ ]:


# Percorsi per le immagini GAN e Stable Diffusion (tutte considerate "generate")
gan_images_dir_64 = '/kaggle/working/Immagini_gan/64x64'
gan_images_dir_256 = '/kaggle/working/Immagini_gan/256x256'
stable_images_dir_64 = '/kaggle/working/Immagini_stable/64x64'
stable_images_dir_256 = '/kaggle/working/Immagini_stable/256x256'

# Percorsi per salvare i risultati IS e FIS
results_dir = '/kaggle/working/IS_FIS_results'
os.makedirs(results_dir, exist_ok=True)

# Dispositivo GPU/CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Percorso del dataset COCO (sostituisci con il percorso corretto)
coco_annotation_file = '/kaggle/input/coco-2017-dataset/coco2017/annotations/instances_train2017.json'
coco_images_dir = '/kaggle/input/coco-2017-dataset/coco2017/train2017'  # Directory delle immagini COCO

# Funzione per caricare immagini dal dataset COCO
def load_coco_images(coco_annotation_file, coco_images_dir, start_idx, end_idx, image_size):
    """
    Carica un insieme di immagini dal dataset COCO tra gli indici specificati, ridimensionandole a una data dimensione.
    
    Args:
    - coco_annotation_file (str): Percorso al file JSON delle annotazioni COCO.
    - coco_images_dir (str): Directory delle immagini COCO.
    - start_idx (int): Indice iniziale delle immagini da caricare.
    - end_idx (int): Indice finale delle immagini da caricare.
    - image_size (int): Dimensione a cui ridimensionare le immagini.

    Returns:
    - list: Lista di immagini caricate e ridimensionate.
    """
    with open(coco_annotation_file, 'r') as f:
        coco_data = json.load(f)
    
    image_ids = [img['file_name'] for img in coco_data['images'][start_idx:end_idx]][:1000]
    images = []
    
    for img_id in image_ids:
        img_path = os.path.join(coco_images_dir, img_id)
        img = Image.open(img_path).convert("RGB")
        img = img.resize((image_size, image_size))
        images.append(img)
    
    return images

# Funzione per caricare immagini generate da una directory (limita a 1000 immagini)
def load_images_from_dir(directory, image_size, limit=1000):
    """
    Carica le immagini generate da una directory e le ridimensiona a una data dimensione.
    
    Args:
    - directory (str): Directory da cui caricare le immagini generate.
    - image_size (int): Dimensione a cui ridimensionare le immagini.
    - limit (int): Numero massimo di immagini da caricare.

    Returns:
    - list: Lista di immagini caricate e ridimensionate.
    """
    images = []
    for filename in os.listdir(directory)[:limit]:
        if filename.endswith(".png") or filename.endswith(".jpg"):
            img_path = os.path.join(directory, filename)
            img = Image.open(img_path).convert("RGB")
            img = img.resize((image_size, image_size))
            images.append(img)
    return images

# Preprocessing per InceptionV3 (resize to 299x299) e ResNet (resize alle dimensioni originali)
def preprocess_images(images, image_size, for_inception=False):
    """
    Preprocessa un insieme di immagini per essere utilizzate in un modello di deep learning.
    
    Args:
    - images (list): Lista di immagini da preprocessare.
    - image_size (int): Dimensione a cui ridimensionare le immagini.
    - for_inception (bool): Se True, ridimensiona le immagini a 299x299 per InceptionV3.

    Returns:
    - Tensor: Batch di immagini preprocessate.
    """
    if for_inception:
        preprocess = transforms.Compose([
            transforms.Resize(299),
            transforms.CenterCrop(299),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ])
    else:
        preprocess = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ])
    
    return torch.stack([preprocess(img) for img in images]).to(device)

# Funzione per calcolare l'Inception Score (IS)
def calculate_inception_score(images, splits=10):
    """
    Calcola l'Inception Score per un insieme di immagini.
    
    Args:
    - images (Tensor): Batch di immagini preprocessate.
    - splits (int): Numero di suddivisioni per calcolare IS.

    Returns:
    - tuple: Media e deviazione standard dell'Inception Score.
    """
    inception_model = models.inception_v3(pretrained=True, transform_input=False).to(device)
    inception_model.eval()

    def get_pred(x):
        x = inception_model(x)
        return torch.nn.functional.softmax(x, dim=1).data.cpu().numpy()

    N = len(images)
    batch_size = 32
    preds = np.zeros((N, 1000))

    for i in range(0, N, batch_size):
        batch_images = images[i:i + batch_size]
        preds[i:i + batch_size] = get_pred(batch_images)

    scores = []
    for i in range(splits):
        part = preds[i * (N // splits): (i + 1) * (N // splits), :]
        py = np.mean(part, axis=0)
        kl_div = part * (np.log(part) - np.log(py))
        kl_div = np.sum(kl_div, axis=1)
        scores.append(np.exp(np.mean(kl_div)))

    return np.mean(scores), np.std(scores)

# Funzione per estrarre le feature dalle immagini usando ResNet18
def extract_features(images, model):
    """
    Estrae le feature da un batch di immagini utilizzando un modello pre-addestrato.
    
    Args:
    - images (Tensor): Batch di immagini preprocessate.
    - model (torch.nn.Module): Modello pre-addestrato per l'estrazione delle feature.

    Returns:
    - Tensor: Feature estratte dalle immagini.
    """
    with torch.no_grad():
        features = model(images)
    return features

# Funzione per calcolare il Feature Importance Score (FIS)
def calculate_fis(real_images, generated_images, model, batch_size=32):
    """
    Calcola il Feature Importance Score (FIS) tra immagini reali e generate.
    
    Args:
    - real_images (Tensor): Batch di immagini reali.
    - generated_images (Tensor): Batch di immagini generate.
    - model (torch.nn.Module): Modello pre-addestrato per l'estrazione delle feature.
    - batch_size (int): Dimensione della batch per l'elaborazione.

    Returns:
    - float: Punteggio medio del FIS.
    """
    total_distance = 0.0
    num_batches = 0

    for i in range(0, len(real_images), batch_size):
        real_batch = real_images[i:i + batch_size]
        generated_batch = generated_images[i:i + batch_size]
        
        real_features = extract_features(real_batch, model)
        generated_features = extract_features(generated_batch, model)
        
        distance = torch.norm(real_features - generated_features, dim=1).mean().item()
        total_distance += distance
        num_batches += 1

    return total_distance / num_batches

# Funzione per salvare il grafico FIS e i risultati finali
def save_fis_results(fis_score, fis_txt_file, fis_plot_file):
    """
    Salva i risultati del Feature Importance Score (FIS) in un file di testo e un grafico.
    
    Args:
    - fis_score (float): Valore del punteggio FIS.
    - fis_txt_file (str): Percorso del file di testo per salvare il punteggio.
    - fis_plot_file (str): Percorso del file per salvare il grafico FIS.
    """
    with open(fis_txt_file, 'w') as f:
        f.write(f"Feature Importance Score (FIS): {fis_score:.4f}\n")

    plt.figure(figsize=(6, 4))
    plt.plot([1], [fis_score], marker='o', linestyle='-', color='b')
    plt.title("Feature Importance Score (FIS) Finale")
    plt.xlabel("Valore Finale")
    plt.ylabel("Feature Importance Score (FIS)")
    plt.grid(True)

    plt.savefig(fis_plot_file)
    plt.close()

# Funzione principale per calcolare IS e FIS
def calculate_is_and_fis(image_size, gan_dir, stable_dir, results_prefix, batch_size=32):
    """
    Calcola l'Inception Score (IS) e il Feature Importance Score (FIS) per immagini reali e generate.
    
    Args:
    - image_size (int): Dimensione delle immagini da utilizzare.
    - gan_dir (str): Directory delle immagini GAN generate.
    - stable_dir (str): Directory delle immagini Stable Diffusion generate.
    - results_prefix (str): Prefisso per i file di risultati.
    - batch_size (int): Dimensione della batch per l'elaborazione.

    Returns:
    - None
    """
    # Caricamento delle immagini reali dal dataset COCO
    real_images = load_coco_images(coco_annotation_file, coco_images_dir, 2000, 4000, image_size)
    real_images_tensor = preprocess_images(real_images[:1000], image_size)  # Usa solo 1000 immagini reali

    # Caricamento delle immagini generate (GAN e Stable)
    gan_images = load_images_from_dir(gan_dir, image_size, limit=1000)
    stable_images = load_images_from_dir(stable_dir, image_size, limit=1000)
    
    # Unisci tutte le immagini generate (GAN + Stable)
    generated_images = gan_images + stable_images
    generated_images_tensor = preprocess_images(generated_images[:1000], image_size)  # Usa solo 1000 immagini generate

    # Inception Score per immagini generate
    generated_images_tensor_299 = preprocess_images(generated_images[:1000], image_size, for_inception=True)  # Resize a 299x299
    mean_is_gan, std_is_gan = calculate_inception_score(generated_images_tensor_299)
    print(f"Inception Score Generated ({image_size}x{image_size}): Mean = {mean_is_gan}, Std = {std_is_gan}")

    # Salva i risultati IS in un file di testo
    is_results_file = os.path.join(results_dir, f'{results_prefix}_inception_score_results.txt')
    with open(is_results_file, 'w') as f:
        f.write(f"Inception Score ({image_size}x{image_size}): Mean = {mean_is_gan}, Std = {std_is_gan}\n")

    # Caricamento del modello ResNet18 per calcolare FIS
    resnet_model = models.resnet18(pretrained=True).eval().to(device)
    resnet_model = nn.Sequential(*list(resnet_model.children())[:-1])  # Rimuovi il layer di classificazione

    # Feature Importance Score (FIS) per immagini generate (considerando tutte le batch)
    fis_gan = calculate_fis(real_images_tensor, generated_images_tensor, resnet_model, batch_size=batch_size)
    print(f"Feature Importance Score Generated ({image_size}x{image_size}): {fis_gan}")

    # Salva i risultati FIS
    fis_results_file = os.path.join(results_dir, f'{results_prefix}_fis_results.txt')
    fis_plot_file = os.path.join(results_dir, f'{results_prefix}_fis_plot.png')
    save_fis_results(fis_gan, fis_results_file, fis_plot_file)

# Calcolo per 64x64 (GAN e Stable Diffusion come generate, COCO come reali)
calculate_is_and_fis(64, gan_images_dir_64, stable_images_dir_64, "64x64", batch_size=16)

# Calcolo per 256x256 (GAN e Stable Diffusion come generate, COCO come reali)
calculate_is_and_fis(256, gan_images_dir_256, stable_images_dir_256, "256x256", batch_size=8)


In [ ]:


# Configurazione per l'uso di GPU o CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ngpu = torch.cuda.device_count()

# Carica il modello VGG16 pre-addestrato
vgg16 = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
vgg16.classifier = vgg16.classifier[:-1]  # Rimuovi l'ultimo strato (strato di classificazione finale)

# Se sono disponibili più GPU, usa DataParallel
if ngpu > 1:
    vgg16 = torch.nn.DataParallel(vgg16)

vgg16.eval().to(device)  # Imposta il modello in modalità di valutazione e spostalo su GPU/CPU

# Funzione per preprocessare le immagini (64x64 o 256x256)
def preprocess_image(image, size=64):
    """
    Preprocessa un'immagine ridimensionandola e normalizzandola.
    
    Args:
    - image (PIL.Image): Immagine da preprocessare.
    - size (int): Dimensione a cui ridimensionare l'immagine (default 64).
    
    Returns:
    - Tensor: Immagine preprocessata come Tensor.
    """
    transform = transforms.Compose([
        transforms.Resize((size, size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return transform(image)

# Dataset personalizzato per caricare le immagini da una cartella
class ImageDataset(Dataset):
    """
    Classe per creare un dataset personalizzato che carica immagini da una directory e le preprocessa.
    
    Args:
    - image_dir (str): Percorso alla directory delle immagini.
    - size (int): Dimensione a cui ridimensionare le immagini.
    """
    def __init__(self, image_dir, size):
        self.image_dir = image_dir
        self.size = size
        self.image_names = [f for f in os.listdir(image_dir) if f.endswith('.jpg') or f.endswith('.png')]

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_names[idx])
        image = Image.open(img_path).convert('RGB')
        image = preprocess_image(image, self.size)
        return image, self.image_names[idx]

# Funzione per estrarre le caratteristiche con VGG16
def extract_features(images, model):
    """
    Estrae le caratteristiche delle immagini utilizzando il modello VGG16.
    
    Args:
    - images (Tensor): Batch di immagini preprocessate.
    - model (torch.nn.Module): Modello pre-addestrato (VGG16) per l'estrazione delle caratteristiche.
    
    Returns:
    - np.ndarray: Caratteristiche estratte dalle immagini.
    """
    images = images.to(device)
    with torch.no_grad():  # Disabilita il calcolo dei gradienti per risparmiare memoria
        features = model(images)
    return features.flatten(1).cpu().numpy()

# Funzione per calcolare e salvare le metriche
def calculate_metrics(y_true, y_pred, save_path):
    """
    Calcola l'accuratezza, precisione, richiamo e F1 score e salva i risultati in un file.
    
    Args:
    - y_true (list): Lista dei valori veri (reali).
    - y_pred (list): Lista delle predizioni (generate dal modello).
    - save_path (str): Percorso del file dove salvare i risultati.
    """
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    # Salva i risultati in un file di testo
    with open(save_path, 'w') as f:
        f.write(f"Accuracy: {accuracy:.4f}\n")
        f.write(f"Precision: {precision:.4f}\n")
        f.write(f"Recall: {recall:.4f}\n")
        f.write(f"F1 Score: {f1:.4f}\n")

    print(f"Metrics saved to {save_path}")

# Funzione per processare le immagini in batch
def process_images_in_batches(dataloader_real, dataloader_generated, model, svm, result_file):
    """
    Processa le immagini reali e generate in batch, estrae le caratteristiche e confronta le predizioni
    utilizzando un classificatore SVM.

    Args:
    - dataloader_real (DataLoader): Dataloader per le immagini reali.
    - dataloader_generated (DataLoader): Dataloader per le immagini generate.
    - model (torch.nn.Module): Modello pre-addestrato per l'estrazione delle caratteristiche.
    - svm (sklearn.svm.SVC): Classificatore SVM per confrontare le caratteristiche.
    - result_file (str): Percorso del file dove salvare i risultati delle metriche.
    """
    y_true = []
    y_pred = []

    for real_batch, _ in dataloader_real:
        real_batch = real_batch.to(device)
        real_features = extract_features(real_batch, model)

        for gen_batch, _ in dataloader_generated:
            gen_batch = gen_batch.to(device)
            gen_features = extract_features(gen_batch, model)

            # Combina le caratteristiche reali e generate
            combined_features = torch.cat([torch.tensor(real_features), torch.tensor(gen_features)], dim=0).numpy()

            # Predizioni con il classificatore SVM
            predictions = svm.predict(combined_features)

            # Aggiungi i valori veri (1 per reale, 0 per generato) e le predizioni
            y_true.extend([1] * len(real_batch) + [0] * len(gen_batch))
            y_pred.extend(predictions)

            # Libera la memoria GPU dopo ogni batch
            torch.cuda.empty_cache()

    calculate_metrics(y_true, y_pred, result_file)

# Percorsi per i checkpoint SVM
svm_checkpoint_64 = '/kaggle/input/checkpoint-classificatore-6464/svm_classifier_64.pkl'

# Percorsi per le immagini ridimensionate
coco_dir = '/kaggle/input/coco-2017-dataset/coco2017/train2017'
stable_64x64_dir = '/kaggle/input/immagini-gan/kaggle/working/Immagini_gan/64x64'
gan_64x64_dir = '/kaggle/input/immagini-gan/kaggle/working/Immagini_gan/64x64'

# Batch size ridotto per ridurre l'uso della memoria GPU
batch_size = 128  # Ridotto per evitare errori di memoria

# Dataset e DataLoader per le immagini COCO 64x64 (limite a 1000 immagini)
dataset_coco_64 = ImageDataset(coco_dir, 64)
dataset_coco_64 = Subset(dataset_coco_64, range(2000))
dataloader_coco_64 = DataLoader(dataset_coco_64, batch_size=batch_size, shuffle=False)

# Dataset e DataLoader per le immagini Stable 64x64 (limite a 1000 immagini)
dataset_stable_64 = ImageDataset(stable_64x64_dir, 64)
dataset_stable_64 = Subset(dataset_stable_64, range(2000))
dataloader_stable_64 = DataLoader(dataset_stable_64, batch_size=batch_size, shuffle=False)

# Dataset e DataLoader per le immagini GAN 64x64 (limite a 1000 immagini)
dataset_gan_64 = ImageDataset(gan_64x64_dir, 64)
dataset_gan_64 = Subset(dataset_gan_64, range(2000))
dataloader_gan_64 = DataLoader(dataset_gan_64, batch_size=batch_size, shuffle=False)

# Carica il classificatore SVM per 64x64
svm_64 = joblib.load(svm_checkpoint_64)

# Confronto tra COCO e Stable Diffusion (64x64)
print("Processing COCO vs Stable Diffusion (64x64)...")
process_images_in_batches(dataloader_coco_64, dataloader_stable_64, vgg16, svm_64, '/kaggle/working/metrics_coco_stable_6464.txt')

# Confronto tra COCO e GAN (64x64)
print("Processing COCO vs GAN (64x64)...")
process_images_in_batches(dataloader_coco_64, dataloader_gan_64, vgg16, svm_64, '/kaggle/working/metrics_coco_gan_6464.txt')

print("Processing complete.")


In [ ]:


# Configurazione per l'uso di GPU o CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ngpu = torch.cuda.device_count()

# Carica il modello VGG16 pre-addestrato
vgg16 = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
vgg16.classifier = vgg16.classifier[:-1]  # Rimuovi l'ultimo strato di classificazione

# Se ci sono più GPU, utilizza DataParallel per parallelizzare il carico di lavoro
if ngpu > 1:
    vgg16 = torch.nn.DataParallel(vgg16)
vgg16.eval().to(device)  # Imposta il modello in modalità di valutazione e spostalo su GPU/CPU

# Funzione per preprocessare le immagini (256x256)
def preprocess_image(image, size=256):
    """
    Preprocessa un'immagine ridimensionandola a una dimensione specificata e normalizzandola.
    
    Args:
    - image (PIL.Image): Immagine da preprocessare.
    - size (int): Dimensione a cui ridimensionare l'immagine (default: 256).
    
    Returns:
    - Tensor: Immagine preprocessata come Tensor.
    """
    transform = transforms.Compose([
        transforms.Resize((size, size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return transform(image)

# Dataset personalizzato per caricare le immagini da una cartella
class ImageDataset(Dataset):
    """
    Classe per creare un dataset personalizzato che carica immagini da una directory e le preprocessa.
    
    Args:
    - image_dir (str): Percorso alla directory delle immagini.
    - size (int): Dimensione a cui ridimensionare le immagini.
    """
    def __init__(self, image_dir, size):
        self.image_dir = image_dir
        self.size = size
        self.image_names = [f for f in os.listdir(image_dir) if f.endswith('.jpg') or f.endswith('.png')]

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_names[idx])
        image = Image.open(img_path).convert('RGB')
        image = preprocess_image(image, self.size)
        return image, self.image_names[idx]

# Funzione per estrarre le caratteristiche con VGG16
def extract_features(images, model):
    """
    Estrae le caratteristiche dalle immagini utilizzando il modello VGG16.
    
    Args:
    - images (Tensor): Batch di immagini preprocessate.
    - model (torch.nn.Module): Modello pre-addestrato per l'estrazione delle caratteristiche (VGG16).
    
    Returns:
    - np.ndarray: Caratteristiche estratte dalle immagini.
    """
    images = images.to(device)
    with torch.no_grad():  # Disabilita il calcolo dei gradienti per risparmiare memoria
        features = model(images)
    return features.flatten(1).cpu().numpy()

# Funzione per calcolare e salvare le metriche
def calculate_metrics(y_true, y_pred, save_path):
    """
    Calcola l'accuratezza, precisione, richiamo e F1 score e salva i risultati in un file di testo.
    
    Args:
    - y_true (list): Lista dei valori reali (1 per reale, 0 per generato).
    - y_pred (list): Lista delle predizioni effettuate dal modello.
    - save_path (str): Percorso del file dove salvare i risultati.
    """
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    # Salva i risultati in un file di testo
    with open(save_path, 'w') as f:
        f.write(f"Accuracy: {accuracy:.4f}\n")
        f.write(f"Precision: {precision:.4f}\n")
        f.write(f"Recall: {recall:.4f}\n")
        f.write(f"F1 Score: {f1:.4f}\n")

    print(f"Metrics saved to {save_path}")

# Funzione per processare le immagini in batch
def process_images_in_batches(dataloader_real, dataloader_generated, model, svm, result_file):
    """
    Processa le immagini reali e generate in batch, estrae le caratteristiche e confronta le predizioni
    utilizzando un classificatore SVM.

    Args:
    - dataloader_real (DataLoader): Dataloader per le immagini reali.
    - dataloader_generated (DataLoader): Dataloader per le immagini generate.
    - model (torch.nn.Module): Modello pre-addestrato per l'estrazione delle caratteristiche.
    - svm (sklearn.svm.SVC): Classificatore SVM per confrontare le caratteristiche.
    - result_file (str): Percorso del file dove salvare i risultati delle metriche.
    """
    y_true = []
    y_pred = []

    for real_batch, _ in dataloader_real:
        real_batch = real_batch.to(device)
        real_features = extract_features(real_batch, model)

        for gen_batch, _ in dataloader_generated:
            gen_batch = gen_batch.to(device)
            gen_features = extract_features(gen_batch, model)

            # Combina le caratteristiche reali e generate
            combined_features = torch.cat([torch.tensor(real_features), torch.tensor(gen_features)], dim=0).numpy()

            # Predizioni con il classificatore SVM
            predictions = svm.predict(combined_features)

            # Aggiungi i valori veri (1 per reale, 0 per generato) e le predizioni
            y_true.extend([1] * len(real_batch) + [0] * len(gen_batch))
            y_pred.extend(predictions)

            # Libera la memoria GPU dopo ogni batch
            torch.cuda.empty_cache()

    calculate_metrics(y_true, y_pred, result_file)

# Percorsi per i checkpoint SVM
svm_checkpoint_256 = '/kaggle/input/checkpoint-classificatore-256256/svm_classifier_256.pkl'

# Percorsi per le immagini ridimensionate
coco_dir = '/kaggle/input/coco-2017-dataset/coco2017/train2017'
stable_256x256_dir = '/kaggle/input/immagini-stable/kaggle/working/Immagini_stable/256x256'
gan_256x256_dir = '/kaggle/input/immagini-gan/kaggle/working/Immagini_gan/256x256'

# Batch size per l'uso ottimale della memoria GPU
batch_size = 128

# Dataset e DataLoader per le immagini COCO 256x256 (limite a 2000 immagini)
dataset_coco_256 = ImageDataset(coco_dir, 256)
dataset_coco_256 = Subset(dataset_coco_256, range(2000))
dataloader_coco_256 = DataLoader(dataset_coco_256, batch_size=batch_size, shuffle=False)

# Dataset e DataLoader per le immagini Stable 256x256 (limite a 2000 immagini)
dataset_stable_256 = ImageDataset(stable_256x256_dir, 256)
dataset_stable_256 = Subset(dataset_stable_256, range(2000))
dataloader_stable_256 = DataLoader(dataset_stable_256, batch_size=batch_size, shuffle=False)

# Dataset e DataLoader per le immagini GAN 256x256 (limite a 2000 immagini)
dataset_gan_256 = ImageDataset(gan_256x256_dir, 256)
dataset_gan_256 = Subset(dataset_gan_256, range(2000))
dataloader_gan_256 = DataLoader(dataset_gan_256, batch_size=batch_size, shuffle=False)

# Carica il classificatore SVM per 256x256
svm_256 = joblib.load(svm_checkpoint_256)

# Confronto tra COCO e Stable Diffusion (256x256)
print("Processing COCO vs Stable Diffusion (256x256)...")
process_images_in_batches(dataloader_coco_256, dataloader_stable_256, vgg16, svm_256, '/kaggle/working/metrics_coco_stable_256256.txt')

# Confronto tra COCO e GAN (256x256)
print("Processing COCO vs GAN (256x256)...")
process_images_in_batches(dataloader_coco_256, dataloader_gan_256, vgg16, svm_256, '/kaggle/working/metrics_coco_gan_256256.txt')

print("Processing complete.")


In [ ]:


# Configurazione per l'uso di GPU o CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ngpu = torch.cuda.device_count()

# Carica il modello VGG16 pre-addestrato
vgg16 = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
vgg16.classifier = vgg16.classifier[:-1]  # Rimuovi l'ultimo strato di classificazione

# Se ci sono più GPU, utilizza DataParallel per parallelizzare il carico di lavoro
if ngpu > 1:
    vgg16 = torch.nn.DataParallel(vgg16)
vgg16.eval().to(device)  # Imposta il modello in modalità di valutazione e spostalo su GPU/CPU

# Carica il classificatore SVM
svm_checkpoint = '/kaggle/input/checkpoint-classificatore-6464/svm_classifier_64.pkl'
svm_classifier = joblib.load(svm_checkpoint)

# Funzione per preprocessare le immagini (64x64)
def preprocess_image(image, size=64):
    """
    Preprocessa un'immagine ridimensionandola a una dimensione specificata e normalizzandola.
    
    Args:
    - image (PIL.Image): Immagine da preprocessare.
    - size (int): Dimensione a cui ridimensionare l'immagine (default 64).
    
    Returns:
    - Tensor: Immagine preprocessata come Tensor.
    """
    transform = transforms.Compose([
        transforms.Resize((size, size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return transform(image)

# Dataset personalizzato per caricare le immagini da una cartella
class ImageDataset(Dataset):
    """
    Classe per creare un dataset personalizzato che carica immagini da una directory e le preprocessa.
    
    Args:
    - image_dir (str): Percorso alla directory delle immagini.
    - size (int): Dimensione a cui ridimensionare le immagini.
    """
    def __init__(self, image_dir, size):
        self.image_dir = image_dir
        self.size = size
        self.image_names = [f for f in os.listdir(image_dir) if f.endswith('.jpg') or f.endswith('.png')]

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_names[idx])
        image = Image.open(img_path).convert('RGB')
        image = preprocess_image(image, self.size)
        return image, self.image_names[idx]

# Funzione per estrarre le caratteristiche con VGG16
def extract_features(images, model):
    """
    Estrae le caratteristiche dalle immagini utilizzando il modello VGG16.
    
    Args:
    - images (Tensor): Batch di immagini preprocessate.
    - model (torch.nn.Module): Modello pre-addestrato per l'estrazione delle caratteristiche (VGG16).
    
    Returns:
    - np.ndarray: Caratteristiche estratte dalle immagini.
    """
    images = images.to(device)
    with torch.no_grad():  # Disabilita il calcolo dei gradienti per risparmiare memoria
        features = model(images)
    return features.flatten(1).cpu().numpy()

# Funzione per calcolare e salvare le metriche
def calculate_metrics(y_true, y_pred, save_path):
    """
    Calcola l'accuratezza, precisione, richiamo e F1 score e salva i risultati in un file di testo.
    
    Args:
    - y_true (list): Lista dei valori reali (1 per reale, 0 per generato).
    - y_pred (list): Lista delle predizioni effettuate dal modello.
    - save_path (str): Percorso del file dove salvare i risultati.
    """
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    # Salva i risultati in un file di testo
    with open(save_path, 'w') as f:
        f.write(f"Accuracy: {accuracy:.4f}\n")
        f.write(f"Precision: {precision:.4f}\n")
        f.write(f"Recall: {recall:.4f}\n")
        f.write(f"F1 Score: {f1:.4f}\n")

    print(f"Metrics saved to {save_path}")

# Funzione per processare le immagini e calcolare le metriche
def process_images(real_loader, stable_loader, gan_loader, model, classifier, result_file):
    """
    Processa le immagini reali, Stable Diffusion e GAN, estrae le caratteristiche e confronta le predizioni
    utilizzando un classificatore SVM.

    Args:
    - real_loader (DataLoader): Dataloader per le immagini reali.
    - stable_loader (DataLoader): Dataloader per le immagini Stable Diffusion.
    - gan_loader (DataLoader): Dataloader per le immagini GAN.
    - model (torch.nn.Module): Modello pre-addestrato per l'estrazione delle caratteristiche.
    - classifier (sklearn.svm.SVC): Classificatore SVM per confrontare le caratteristiche.
    - result_file (str): Percorso del file dove salvare i risultati delle metriche.
    """
    y_true = []
    y_pred = []

    # Processa immagini reali
    for real_batch, _ in real_loader:
        real_batch = real_batch.to(device)
        real_features = extract_features(real_batch, model)
        predictions_real = classifier.predict(real_features)
        y_true.extend([1] * len(real_batch))  # Label 1 per immagini reali
        y_pred.extend(predictions_real)

    # Processa immagini generate da Stable Diffusion
    for stable_batch, _ in stable_loader:
        stable_batch = stable_batch.to(device)
        stable_features = extract_features(stable_batch, model)
        predictions_stable = classifier.predict(stable_features)
        y_true.extend([0] * len(stable_batch))  # Label 0 per immagini generate
        y_pred.extend(predictions_stable)

    # Processa immagini generate da GAN
    for gan_batch, _ in gan_loader:
        gan_batch = gan_batch.to(device)
        gan_features = extract_features(gan_batch, model)
        predictions_gan = classifier.predict(gan_features)
        y_true.extend([0] * len(gan_batch))  # Label 0 per immagini generate
        y_pred.extend(predictions_gan)

    # Calcola le metriche
    calculate_metrics(y_true, y_pred, result_file)

# Percorsi per le immagini
coco_dir = '/kaggle/input/coco-2017-dataset/coco2017/train2017'
stable_64x64_dir = '/kaggle/input/immagini-stable/kaggle/working/Immagini_stable/64x64'
gan_64x64_dir = '/kaggle/input/immagini-gan/kaggle/working/Immagini_gan/64x64'

# Batch size
batch_size = 128

# Dataset e DataLoader per le immagini COCO 64x64 (2000 immagini)
dataset_coco_64 = ImageDataset(coco_dir, 64)
dataset_coco_64 = Subset(dataset_coco_64, range(2000))
dataloader_coco_64 = DataLoader(dataset_coco_64, batch_size=batch_size, shuffle=False)

# Dataset e DataLoader per le immagini Stable 64x64 (1000 immagini)
dataset_stable_64 = ImageDataset(stable_64x64_dir, 64)
dataset_stable_64 = Subset(dataset_stable_64, range(1000))
dataloader_stable_64 = DataLoader(dataset_stable_64, batch_size=batch_size, shuffle=False)

# Dataset e DataLoader per le immagini GAN 64x64 (1000 immagini)
dataset_gan_64 = ImageDataset(gan_64x64_dir, 64)
dataset_gan_64 = Subset(dataset_gan_64, range(1000))
dataloader_gan_64 = DataLoader(dataset_gan_64, batch_size=batch_size, shuffle=False)

# Risultati e directory per le metriche
metrics_file = '/kaggle/working/metrics_combined_64x64.txt'

# Esegui il calcolo e salva i risultati
print("Processing and calculating metrics...")
process_images(dataloader_coco_64, dataloader_stable_64, dataloader_gan_64, vgg16, svm_classifier, metrics_file)

print("Processing complete.")


In [ ]:


# Configurazione per l'uso di GPU o CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ngpu = torch.cuda.device_count()

# Carica il modello VGG16 pre-addestrato
vgg16 = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
vgg16.classifier = vgg16.classifier[:-1]  # Rimuovi l'ultimo strato di classificazione

# Se ci sono più GPU, utilizza DataParallel per parallelizzare il carico di lavoro
if ngpu > 1:
    vgg16 = torch.nn.DataParallel(vgg16)
vgg16.eval().to(device)  # Imposta il modello in modalità di valutazione e spostalo su GPU/CPU

# Funzione per preprocessare e comprimere le immagini (64x64)
def preprocess_and_compress_image(image, size=64, compression_quality=100):  # Compressione, 100 = nessuna compressione
    """
    Preprocessa e comprime un'immagine ridimensionandola a una dimensione specificata, poi la salva con un livello
    di compressione specifico e la ricarica per ulteriori operazioni.

    Args:
    - image (PIL.Image): Immagine da preprocessare.
    - size (int): Dimensione a cui ridimensionare l'immagine (default 64).
    - compression_quality (int): Qualità della compressione JPEG (100 = nessuna compressione).

    Returns:
    - Tensor: Immagine preprocessata e compressa.
    """
    transform = transforms.Compose([
        transforms.Resize((size, size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Comprime l'immagine utilizzando il formato JPEG con la qualità specificata
    temp_img_path = '/kaggle/working/temp_image.jpg'
    image = image.resize((size, size))  # Ridimensiona l'immagine
    image.save(temp_img_path, format='JPEG', quality=compression_quality)  # Salva con la qualità specificata

    # Ricarica l'immagine compressa e preprocessala
    compressed_image = Image.open(temp_img_path).convert('RGB')
    return transform(compressed_image)

# Dataset personalizzato per caricare le immagini da una cartella e applicare la compressione
class ImageDataset(Dataset):
    """
    Classe per creare un dataset personalizzato che carica immagini da una directory e applica
    la compressione e il preprocessing.

    Args:
    - image_dir (str): Percorso alla directory delle immagini.
    - size (int): Dimensione a cui ridimensionare le immagini.
    - compression_quality (int): Qualità della compressione JPEG.
    """
    def __init__(self, image_dir, size, compression_quality=100):
        self.image_dir = image_dir
        self.size = size
        self.compression_quality = compression_quality
        self.image_names = [f for f in os.listdir(image_dir) if f.endswith('.jpg') or f.endswith('.png')]

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_names[idx])
        image = Image.open(img_path).convert('RGB')
        image = preprocess_and_compress_image(image, self.size, self.compression_quality)
        return image, self.image_names[idx]

# Funzione per estrarre le caratteristiche con VGG16
def extract_features(images, model):
    """
    Estrae le caratteristiche dalle immagini utilizzando il modello VGG16.

    Args:
    - images (Tensor): Batch di immagini preprocessate.
    - model (torch.nn.Module): Modello pre-addestrato per l'estrazione delle caratteristiche.

    Returns:
    - np.ndarray: Caratteristiche estratte dalle immagini.
    """
    images = images.to(device)
    with torch.no_grad():  # Disabilita il calcolo dei gradienti per risparmiare memoria
        features = model(images)
    return features.flatten(1).cpu().numpy()

# Funzione per calcolare e salvare le metriche
def calculate_metrics(y_true, y_pred, save_path):
    """
    Calcola l'accuratezza, precisione, richiamo e F1 score e salva i risultati in un file di testo.

    Args:
    - y_true (list): Lista dei valori reali (1 per reale, 0 per generato).
    - y_pred (list): Lista delle predizioni effettuate dal modello.
    - save_path (str): Percorso del file dove salvare i risultati.
    """
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    # Salva i risultati in un file di testo
    with open(save_path, 'w') as f:
        f.write(f"Accuracy: {accuracy:.4f}\n")
        f.write(f"Precision: {precision:.4f}\n")
        f.write(f"Recall: {recall:.4f}\n")
        f.write(f"F1 Score: {f1:.4f}\n")

    print(f"Metrics saved to {save_path}")

# Funzione per processare le immagini in batch
def process_images_in_batches(dataloader_real, dataloader_generated, model, svm, result_file):
    """
    Processa le immagini reali e generate in batch, estrae le caratteristiche e confronta le predizioni
    utilizzando un classificatore SVM.

    Args:
    - dataloader_real (DataLoader): Dataloader per le immagini reali.
    - dataloader_generated (DataLoader): Dataloader per le immagini generate.
    - model (torch.nn.Module): Modello pre-addestrato per l'estrazione delle caratteristiche.
    - svm (sklearn.svm.SVC): Classificatore SVM per confrontare le caratteristiche.
    - result_file (str): Percorso del file dove salvare i risultati delle metriche.
    """
    y_true = []
    y_pred = []

    for real_batch, _ in dataloader_real:
        real_batch = real_batch.to(device)
        real_features = extract_features(real_batch, model)

        for gen_batch, _ in dataloader_generated:
            gen_batch = gen_batch.to(device)
            gen_features = extract_features(gen_batch, model)

            combined_features = torch.cat([torch.tensor(real_features), torch.tensor(gen_features)], dim=0).numpy()

            predictions = svm.predict(combined_features)

            y_true.extend([1] * len(real_batch) + [0] * len(gen_batch))
            y_pred.extend(predictions)

            # Libera la memoria GPU dopo ogni batch
            torch.cuda.empty_cache()

    calculate_metrics(y_true, y_pred, result_file)

# Percorsi per i checkpoint SVM
svm_checkpoint_64 = '/kaggle/input/checkpoint-classificatore-6464/svm_classifier_64.pkl'

# Percorsi per le immagini ridimensionate
coco_dir = '/kaggle/input/coco-2017-dataset/coco2017/train2017'
stable_64x64_dir = '/kaggle/input/immagini-stable/kaggle/working/Immagini_stable/64x64'
gan_64x64_dir = '/kaggle/input/immagini-gan/kaggle/working/Immagini_gan/64x64'

# Batch size
batch_size = 128

# Livelli di compressione
compression_levels = {
    '40': 60,  # 40% di compressione (60% qualità)
    '60': 40,  # 60% di compressione (40% qualità)
    '80': 20   # 80% di compressione (20% qualità)
}

# Dataset e DataLoader per le immagini COCO 64x64 (limite a 1000 immagini)
dataset_coco_64 = ImageDataset(coco_dir, 64, compression_quality=100)  # Nessuna compressione per COCO
dataset_coco_64 = Subset(dataset_coco_64, range(1000))
dataloader_coco_64 = DataLoader(dataset_coco_64, batch_size=batch_size, shuffle=False)

# Carica il classificatore SVM per 64x64
svm_64 = joblib.load(svm_checkpoint_64)

# Itera sui diversi livelli di compressione
for level, quality in compression_levels.items():
    # Dataset e DataLoader per il dataset misto GAN e Stable (1000 immagini da ciascuno)
    dataset_stable_64 = ImageDataset(stable_64x64_dir, 64, compression_quality=quality)
    dataset_stable_64 = Subset(dataset_stable_64, range(1000))

    dataset_gan_64 = ImageDataset(gan_64x64_dir, 64, compression_quality=quality)
    dataset_gan_64 = Subset(dataset_gan_64, range(1000))

    # Unisci i dataset GAN e Stable in un unico DataLoader
    combined_dataset = dataset_stable_64 + dataset_gan_64
    dataloader_combined = DataLoader(combined_dataset, batch_size=batch_size, shuffle=False)

    # Confronto tra COCO e il dataset misto (64x64 con compressione)
    print(f"Processing COCO vs GAN + Stable Diffusion (64x64, compressione {level}%)...")
    result_file = f'/kaggle/working/metrics_coco_combined_6464_compression_{level}.txt'
    process_images_in_batches(dataloader_coco_64, dataloader_combined, vgg16, svm_64, result_file)

print("Processing complete.")
